In [4]:
import ctypes
import numpy as np
from picosdk.ps4000a import ps4000a as ps
import matplotlib.pyplot as plt
from picosdk.functions import adc2mV, assert_pico_ok
import time
import h5py
import scipy.io as sio
import pico_stream as psu

In [5]:
"""
Set parameters here
"""

# Channel E does not work... apparently it isn't a legit Key...

channels = ['A', 'B', 'C', 'D', 'F', 'G', 'H']
channel_ranges = [7, 7, 7, 7, 7, 7, 7]
analogue_offsets = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

enabled = 1
disabled = 0

# Size of capture
sizeOfOneBuffer = 1000000
numBuffersToCapture = 1
num_files = 10 # number of files tp capture

sample_interval = 1 # in us

totalSamples = sizeOfOneBuffer * numBuffersToCapture
total_length = totalSamples*sample_interval

print('Total length: %s us' % total_length)

downsample_plot = 1000

"""
End of set parameters
"""

Total length: 1000000 us


'\nEnd of set parameters\n'

In [6]:
# Turn on the picoscope
chandle, status = psu.pico_TurnOn()

# Set up the channels
for n in range(len(channels)):
    psu.set_Channel(channels[n], channel_ranges[n], analogue_offsets[n])

# Create buffers ready for assigning pointers for data collection
bufferMaxl = np.zeros(shape=(len(channels), sizeOfOneBuffer), dtype=np.int16)

memory_segment = 0 # Not 100% sure how this affects things but will mess around with it later to understand

# set up the buffer in the picoscope
for n, i in enumerate(channels):
    psu.set_DataBuffer(i, n)

sampleInterval = ctypes.c_int32(sample_interval)
sampleUnits = ps.PS4000A_TIME_UNITS['PS4000A_US']
actualSampleInterval = sampleInterval.value
actualSampleIntervalNs = actualSampleInterval * 1000

# Begin streaming mode:

bufferCompletel = np.zeros(shape=(len(channels), totalSamples), dtype=np.int16)
starts = np.zeros(num_files)
ends = np.zeros(num_files)
for i in range(num_files):
    start = time.time()
    nextSample = 0
    psu.stream()
    maxADC = ctypes.c_int16()
    status["maximumValue"] = ps.ps4000aMaximumValue(chandle, ctypes.byref(maxADC))
    assert_pico_ok(status["maximumValue"])
    adc2mVChlMax = psu.adc2mV2(bufferCompletel, channel_ranges, maxADC)
    psu.save_data('C:/Users/thoma/OneDrive/Documents/SIMPLE/Data/PicoTest/1.hdf5', adc2mVChlMax)
    end = time.time()
    starts[i] = start
    ends[i] = end

# Stop the scope
# handle = chandle
status["stop"] = ps.ps4000aStop(chandle)
assert_pico_ok(status["stop"])

# Disconnect the scope
# handle = chandle
status["close"] = ps.ps4000aCloseUnit(chandle)
assert_pico_ok(status["close"])

# Display status returns
print(status)

{'openunit': 0, 'maximumValue': 0, 'stop': 0, 'close': 0}


In [8]:
ends

array([1.71649684e+09, 1.71649684e+09, 1.71649684e+09, 1.71649684e+09,
       1.71649684e+09, 1.71649684e+09, 1.71649684e+09, 1.71649684e+09,
       1.71649684e+09, 1.71649684e+09])